In [17]:
import pandas as pd
import cv2 
import torch
import torch.nn as nn
import torch.optim as optim 

import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as utils
from torchvision import transforms
import albumentations as A
import plotly.graph_objects as go
import plotly.express as px

In [31]:
data = pd.read_csv('paintings_data.csv')

In [32]:
data.head()

,image_url,material,price,styles,width,length,colorfield,lyrical,punk,obsessive,...,contempory,mural,realism paintings,baroque,geometry,impresionismo,neo-abstraction,new media,popart,primitivo
0,img1.jpg,Glass,2870.0,"Abstract,Figurative,Street Art,Expressionism,C...",32.3,44.1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,img2.jpg,Paper,255.0,Abstract,11.8,9.1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,img3.jpg,Canvas,2540.0,"Abstract,Abstract Expressionism,Figurative,Fin...",27.2,29.1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,img4.jpg,Canvas,4650.0,"Expressionism,Surrealism",55.1,69.3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,img5.jpg,Canvas,3210.0,"Abstract,Modern,Abstract Expressionism",36.0,48.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
data['material'].unique()

array(['Glass', 'Paper', 'Canvas', 'Wood', 'Paper,Wood',
       'Soft (Yarn, Cotton, Fabric)',
       'Canvas,Soft (Yarn, Cotton, Fabric),Other', 'Canvas,Paper',
       'Cardboard', 'Canvas,Wood', 'Soft (Yarn, Cotton, Fabric),Wood',
       'Plastic', 'Canvas,Other', 'Other,Plastic', 'Canvas,Plastic',
       'Other', 'Iron,Plastic,Steel,Other', 'Paper,Canvas',
       'Other,Canvas', 'Canvas,Cardboard,Paper,Wood,Other',
       'Canvas,Soft (Yarn, Cotton, Fabric)', 'Canvas,Cardboard',
       'Cardboard,Paper,Other', 'Aluminium', 'Canvas,Aluminium,Bronze',
       'Canvas,Other,Wood', 'Soft (Yarn, Cotton, Fabric),Canvas',
       'Aluminium,Canvas', 'Cardboard,Canvas', 'Wood,Other',
       'Paper,Wood,Other', 'Paper,Cardboard',
       'Paper,Soft (Yarn, Cotton, Fabric)', 'Canvas,Carbon Fibre',
       'Wood,Canvas,Soft (Yarn, Cotton, Fabric)',
       'Canvas,Cardboard,Other', 'Wood,Canvas', 'Paper,Other',
       'Canvas,Paper,Wood,Other', 'Cardboard,Paper', 'Other,Wood,Paper',
       'Glass,P

In [34]:
str_to_del = '(Yarn, Cotton, Fabric)'
un_materials = []
for i in data.material.unique():
    deleted_str = i.replace(str_to_del, '')
    materials = deleted_str.split(',')
    materials = [material.strip().lower() for material in materials ]
    un_materials += materials
un_materials = set (un_materials)
len(un_materials)

17

In [35]:
un_materials

{'aluminium',
 'bronze',
 'canvas',
 'carbon fibre',
 'cardboard',
 'ceramic',
 'glass',
 'iron',
 'marble',
 'other',
 'paper',
 'plastic',
 'soft',
 'stainless steel',
 'steel',
 'stone',
 'wood'}

In [36]:
for i in un_materials:
    data[i] = 0

In [37]:
def set_materials(dataframe):
    for i in range (len(dataframe)):
        materials = dataframe.loc[i, 'material'].replace(str_to_del, '').split(',')
        materials = [material.strip().lower() for material in materials]
        dataframe.loc[i, materials] = 1
    return dataframe

In [38]:
data = set_materials(data)

In [39]:
data.head()

,image_url,material,price,styles,width,length,colorfield,lyrical,punk,obsessive,...,stone,stainless steel,glass,wood,ceramic,bronze,other,plastic,aluminium,carbon fibre
0,img1.jpg,Glass,2870.0,"Abstract,Figurative,Street Art,Expressionism,C...",32.3,44.1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,img2.jpg,Paper,255.0,Abstract,11.8,9.1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,img3.jpg,Canvas,2540.0,"Abstract,Abstract Expressionism,Figurative,Fin...",27.2,29.1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,img4.jpg,Canvas,4650.0,"Expressionism,Surrealism",55.1,69.3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,img5.jpg,Canvas,3210.0,"Abstract,Modern,Abstract Expressionism",36.0,48.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
fig = go.Figure(data = [
        go.Pie(labels = data.columns[249: ],
           values = data.iloc[:, 249:].sum().values)
])
fig.show()

In [41]:
data.drop(['styles', 'material'], inplace = True, axis = 1)
data.head()

,image_url,price,width,length,colorfield,lyrical,punk,obsessive,spiritual,painting,...,stone,stainless steel,glass,wood,ceramic,bronze,other,plastic,aluminium,carbon fibre
0,img1.jpg,2870.0,32.3,44.1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,img2.jpg,255.0,11.8,9.1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,img3.jpg,2540.0,27.2,29.1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,img4.jpg,4650.0,55.1,69.3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,img5.jpg,3210.0,36.0,48.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
data.columns.get_loc('plastic')

261

In [43]:
fig = go.Figure(data = [
        go.Pie(labels = data.columns[4:248],
           values = data.iloc[:, 4:248].sum().values)
])
fig.show()

In [44]:
fig = px.histogram(data, x="price", histnorm='probability', nbins = 20)
fig.show()

In [46]:
Img_path = 'images/'
idx = 0

image_src = Img_path + data.loc[idx, 'image_url']
image = cv2.imread(image_src, cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
class PaintingsDataset(Dataset):
    
    def __init__(self, df, transforms=None):
    
        self.df = df
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = 'images/' + self.df.loc[idx, 'image_url']
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.loc[idx, 'price'].values
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, labels